In [40]:
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from utils import *
from sklearn.cluster import DBSCAN

In [41]:
df = pd.read_json("/media/influx/Programming/Projects/project-final-random/Linkedin/src/main/resources/users.json")

In [42]:
df_train = df.drop(['dateOfBirth', 'name', 'email', 'profile_pic', 'connectionId'], axis=1)

In [43]:
df_train

,id,universityLocation,field,workplace,specialties
0,1,Tehran,AA,Tesla,"[Database, Front-End, Crypto]"
1,2,Princeton,BB,Apple,"[Full-Stack, Machine Learning, Fast Type]"
2,3,Cambridge,AA,Meta,"[Blockchain, C++, AI]"
3,4,Cambridge,BB,Intel,"[Photoshop, Game, Django]"
4,5,Princeton,DD,AMD,"[Full-Stack, Machine Learning, Photoshop, Game..."
...,...,...,...,...,...
1995,1996,Berkeley,BB,TesUberla,"[Java, Hardware, Verilog, Database, IT, Game]"
1996,1997,ETH,AA,Digikala,"[Verilog, Crypto, C#, Front-End, Full-Stack]"
1997,1998,Carnegie,DD,AMD,"[PHP, Machine Learning, Python, Database, Data..."
1998,1999,Cambridge,AA,AMD,"[Machine Learning, Data, Hardware, Django]"


In [44]:
df_train = pd.get_dummies(df_train, columns=["field"])

In [46]:
df_train = pd.get_dummies(df_train, columns=["universityLocation"])

In [48]:
df_train = pd.get_dummies(df_train, columns=["workplace"])

In [50]:
df_train['specialties']

0                           [Database, Front-End, Crypto]
1               [Full-Stack, Machine Learning, Fast Type]
2                                   [Blockchain, C++, AI]
3                               [Photoshop, Game, Django]
4       [Full-Stack, Machine Learning, Photoshop, Game...
                              ...                        
1995        [Java, Hardware, Verilog, Database, IT, Game]
1996         [Verilog, Crypto, C#, Front-End, Full-Stack]
1997    [PHP, Machine Learning, Python, Database, Data...
1998           [Machine Learning, Data, Hardware, Django]
1999                   [Photoshop, Machine Learning, C++]
Name: specialties, Length: 2000, dtype: object

In [51]:
spec = set(s for user in df_train['specialties'] for s in user)

In [52]:
print(spec)

{'IT', 'Verilog', 'Software', 'PHP', 'Blockchain', 'Hardware', 'Database', 'Machine Learning', 'AI', 'Fast Type', 'C++', 'Back-End', 'Photoshop', 'Data', 'Python', 'Django', 'Front-End', 'C#', 'Crypto', 'Full-Stack', 'Game', 'Java', 'Algorithm'}


In [53]:
for special in spec:
    df_train[special] = [0 for i in range(2000)]

In [54]:
df_train

,id,specialties,field_AA,field_BB,field_CC,field_DD,universityLocation_Amirkabir,universityLocation_Berkeley,universityLocation_Cambridge,universityLocation_Carnegie,...,Data,Python,Django,Front-End,C#,Crypto,Full-Stack,Game,Java,Algorithm
0,1,"[Database, Front-End, Crypto]",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,"[Full-Stack, Machine Learning, Fast Type]",0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,"[Blockchain, C++, AI]",1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,"[Photoshop, Game, Django]",0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,"[Full-Stack, Machine Learning, Photoshop, Game...",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,"[Java, Hardware, Verilog, Database, IT, Game]",0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1996,1997,"[Verilog, Crypto, C#, Front-End, Full-Stack]",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1997,1998,"[PHP, Machine Learning, Python, Database, Data...",0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1998,1999,"[Machine Learning, Data, Hardware, Django]",1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
i = 0
for user in df_train['specialties']:
    for s in user:
        df_train.at[i, s] = 1
    i+=1

In [56]:
df_train = df_train.drop(['specialties', 'id'], axis=1)

In [59]:
df_train

,field_AA,field_BB,field_CC,field_DD,universityLocation_Amirkabir,universityLocation_Berkeley,universityLocation_Cambridge,universityLocation_Carnegie,universityLocation_ETH,universityLocation_Esfahan,...,Data,Python,Django,Front-End,C#,Crypto,Full-Stack,Game,Java,Algorithm
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1996,1,0,0,0,0,0,0,0,1,0,...,0,0,0,1,1,1,1,0,0,0
1997,0,0,0,1,0,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
1998,1,0,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,0,0


In [60]:
X = df_train.values

In [61]:
X

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [62]:
len(X)

2000

In [63]:
wcss = []
for i in range(1, 200):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 10)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

In [64]:
plt.plot(range(1, 200), wcss)
plt.title('The Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [65]:
kmeans = KMeans(n_clusters = 100, init = 'k-means++', random_state = 10)
y_kmeans = kmeans.fit_predict(X)

In [67]:
y_kmeans

array([ 36,  11,  54, ..., 136,  46,  97], dtype=int32)

In [68]:
len(y_kmeans)

2000

In [69]:
idx = 0
o = list()
for l in y_kmeans:
    if l == 13:
        o.append(idx)
    idx += 1

In [70]:
o

[366,
 431,
 489,
 548,
 750,
 782,
 815,
 1106,
 1170,
 1182,
 1262,
 1298,
 1308,
 1328,
 1367,
 1656,
 1758]

In [99]:
users = list()
for idx in o:
    users.append(df.iloc[idx, :])

In [72]:
users

[id                                                                  367
 name                                                         arya Hines
 dateOfBirth                                                    2009/3/4
 universityLocation                                                  MIT
 field                                                                AA
 workplace                                                        Amazon
 specialties                             [Java, PHP, Database, Hardware]
 connectionId          [2000, 1486, 266, 73, 824, 936, 1985, 1087, 19...
 email                                         aryaHines2009@skinner.org
 profile_pic            https://randomuser.me/api/portraits/women/26.jpg
 Name: 366, dtype: object,
 id                                                                  432
 name                                                       farjAd Starr
 dateOfBirth                                                   2008/1/21
 universityLocation     